# From NumPy arrays to Pandas DataFrame

This notebook is adapated from Joris Van den Bossche tutorial:

* https://github.com/paris-saclay-cds/python-workshop/blob/master/Day_1_Scientific_Python/02-pandas_introduction.ipynb

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.options.display.max_rows = 8

# 0. Let's start with a showcase

#### Case 1: titanic survival data

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
df.head()

Starting from reading this dataset, to answering questions about this data in a few lines of code:

**What is the age distribution of the passengers?**

In [ ]:
df['Age'].hist()

**How does the survival rate of the passengers differ between sexes?**

In [ ]:
df.groupby('Sex')[['Survived']].aggregate(lambda x: x.sum() / len(x))

**Or how does it differ between the different classes?**

In [ ]:
df.groupby('Pclass')['Survived'].aggregate(lambda x: x.sum() / len(x)).plot(kind='bar')

All the needed functionality for the above examples will be explained throughout this tutorial.

#### Case 2: air quality measurement timeseries

AirBase (The European Air quality dataBase): hourly measurements of all air quality monitoring stations from Europe

Starting from these hourly data for different stations:

In [ ]:
data = pd.read_csv('data/20000101_20161231-NO2.csv', sep=';', skiprows=[1], na_values=['n/d'], index_col=0, parse_dates=True)

In [ ]:
data.head()

to answering questions about this data in a few lines of code:

**Does the air pollution show a decreasing trend over the years?**

In [ ]:
data['1999':].resample('M').mean().plot(ylim=[0,120])

In [ ]:
data['1999':].resample('A').mean().plot(ylim=[0,100])

**What is the difference in diurnal profile between weekdays and weekend?**

In [ ]:
data['weekday'] = data.index.weekday
data['weekend'] = data['weekday'].isin([5, 6])
data_weekend = data.groupby(['weekend', data.index.hour])['BASCH'].mean().unstack(level=0)
data_weekend.plot()

We will come back to these example, and build them up step by step.

## 1. Pandas: data analysis in python

For data-intensive work in Python the [Pandas](http://pandas.pydata.org) library has become essential.

**What is `pandas`?**

* Pandas can be thought of as *NumPy arrays with labels* for rows and columns, and better support for heterogeneous data types, but it's also much, much more than that.
* Pandas can also be thought of as `R`'s `data.frame` in Python.
* Powerful for working with missing data, working with time series data, for reading and writing your data, for reshaping, grouping, merging your data, ...

It's documentation: http://pandas.pydata.org/pandas-docs/stable/


**When do you need pandas?**

When working with **tabular or structured data** (like R dataframe, SQL table, Excel spreadsheet, ...):

- Import data
- Clean up messy data
- Explore data, gain insight into data
- Process and prepare your data for analysis
- Analyse your data (together with scikit-learn, statsmodels, ...)

<div class="alert alert-warning">
<b>ATTENTION!</b>: <br><br>

Pandas is great for working with heterogeneous and tabular 1D/2D data, but not all types of data fit in such structures!
<ul>
<li>When working with array data (e.g. images, numerical algorithms): just stick with numpy</li>
<li>When working with multidimensional labeled data (e.g. climate data): have a look at [xarray](http://xarray.pydata.org/en/stable/)</li>
</ul>
</div>

## 2. The pandas data structures: `DataFrame` and `Series`

A `DataFrame` is a **tablular data structure** (multi-dimensional object to hold labeled data) comprised of rows and columns, akin to a spreadsheet, database table, or R's data.frame object. You can think of it as multiple Series object which share the same index.


<img align="left" width=50% src="./schema-dataframe.svg">

We can create a pandas Dataframe and specify the index and columns to use.

In [ ]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
df_countries = pd.DataFrame(data)
df_countries

As previously mentioned, the dataframe stores information regarding the column and index information.

In [ ]:
df_countries.columns

In [ ]:
df_countries.index

We previously saw that NumPy arrays store data using an homogeneous type and contiguously in memory. Pandas is used to store heterogeneous data where each column could be of different data types.

In [ ]:
df_countries.dtypes

You can get an overview of the information of a dataframe using the `info()` method:

In [ ]:
df_countries.info()

It is always possible to get a NumPy array, dropping the index and columns information:

In [ ]:
df_countries.values

However, be aware that all values will be upcasted in case of heterogeneous data.

### One-dimensional data: `Series` (a column of a DataFrame)

A Series is a basic holder for **one-dimensional labeled data**.

In [ ]:
df_countries['population']

In [ ]:
population = df_countries['population']

A pandas series as an index as well. Like the dataframe, you can get a NumPy array calling the attribute `values`

In [ ]:
population.values

### Exercise

We will define a set of 1D NumPy arrays containing the data that we will work with.

In [ ]:
country_name = np.array(['Austria', 'Iran, Islamic Rep.', 'France'], dtype=object)
country_code = np.array(['AUT', 'IRN', 'FRA'], dtype=object)
gdp_2015 = np.array([1349034029453.37, 385874474398.59, 2438207896251.84])
gdp_2017 = np.array([1532397555.55556, 439513511620.591,2582501307216.42])

* Create a `DataFrame` by passing a single NumPy array (Hint: concatenate the differen 1D NumPy arrays). In addition, provide a Python list with the name of the columns to the `columns` parameter.

In [ ]:
# %load solutions/01_solutions.py

* Instead of concatenating the data, create a Python dictionary where the keys will be the name of the columns and the values will be the corresponding NumPy array.

In [ ]:
# %load solutions/02_solutions.py

* Use the same procedure (Python dictionary) but specify that the country code should be used as the index.

In [ ]:
# %load solutions/03_solutions.py

In [ ]:
# %load solutions/04_solutions.py

## 3. Selecting and filtering data

<div class="alert alert-warning">
<b>ATTENTION!</b>: <br><br>

One of pandas' basic features is the labeling of rows and columns, but this makes indexing also a bit more complex compared to numpy. <br><br> We now have to distuinguish between:

 <ul>
  <li>selection by **label**</li>
  <li>selection by **position**</li>
</ul>
</div>

For this example, we will reuse the `df_countries` dataset.

In [ ]:
df_countries

### `df[]` provides some convenience shortcuts 

For a DataFrame, basic indexing selects the columns.

Selecting a single column:

In [ ]:
df_countries['population']

or multiple columns:

In [ ]:
df_countries[['country', 'capital']]

But, slicing accesses the rows:

In [ ]:
df_countries[2:5]

### Systematic indexing with `loc` and `iloc`

When using `[]` like above, you can only select from one axis at once (rows or columns, not both). For more advanced indexing, you have some extra attributes:
    
* `loc`: selection by label
* `iloc`: selection by position

In [ ]:
df_countries = df_countries.set_index('country')

In [ ]:
df_countries

In [ ]:
df_countries.loc['France', 'capital']

In [ ]:
df_countries.loc['France':'Netherlands', :]

Selecting by position with `iloc` works similar as indexing numpy arrays:

In [ ]:
df_countries.iloc[0:2,1:3]

The different indexing methods can also be used to assign data:

In [ ]:
df_countries.loc['France', 'population'] = 70

In [ ]:
df_countries

### Boolean indexing (filtering)

Often, you want to select rows based on a certain condition. This can be done with 'boolean indexing' (like a where clause in SQL) and comparable to numpy. 

The indexer (or boolean mask) should be 1-dimensional and the same length as the thing being indexed.

In [ ]:
df_countries['population'] > 60

In [ ]:
df_countries[df_countries['population'] > 60]

### Exercise

In [ ]:
df = pd.read_csv("data/titanic.csv")
df.head()

Select the sub-dataframe for which the men are older than 60 years old. Using the attribute shape, find how many individual correspond to this criteria.

In [ ]:
# %load solutions/05_solutions.py

## 4. Statistical analysis

Pandas provides an easy and fast way to explore data. Let's explore the `titanic` data set.

In [ ]:
df = df.set_index('Name')
df.head()

We will select the `Age` column and compute couple of statistic.

In [ ]:
age = df['Age']
age

In [ ]:
age.mean()

In [ ]:
age.max()

In [ ]:
age.min()

In [ ]:
age.describe()

In [ ]:
age.value_counts()

In [ ]:
age.hist()

### Exercise

* What is the maximum Fare that was paid? And the median?

In [ ]:
# %load solutions/06_solutions.py

In [ ]:
# %load solutions/07_solutions.py

* Calculate the average survival ratio for all passengers (note: the 'Survived' column indicates whether someone survived (1) or not (0)).

In [ ]:
# %load solutions/08_solutions.py

In [ ]:
# %load solutions/09_solutions.py

* Select the sub-dataframe for which the men are older than 60 years old.

In [ ]:
# %load solutions/10_solutions.py

* Based on the titanic data set, select all rows for male passengers and calculate the mean age of those passengers. Do the same for the female passengers.

In [ ]:
# %load solutions/11_solutions.py

In [ ]:
# %load solutions/12_solutions.py

* Plot the Fare distribution.

In [ ]:
# %load solutions/13_solutions.py

## 5. The group-by operation

### Some 'theory': the groupby operation (split-apply-combine)

In [ ]:
df = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                   'data': [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

### Recap: aggregating functions

When analyzing data, you often calculate summary statistics (aggregations like the mean, max, ...). As we have seen before, we can easily calculate such a statistic for a Series or column using one of the many available methods. For example:

In [ ]:
df['data'].sum()

However, in many cases your data has certain groups in it, and in that case, you may want to calculate this statistic for each of the groups.

For example, in the above dataframe `df`, there is a column 'key' which has three possible values: 'A', 'B' and 'C'. When we want to calculate the sum for each of those groups, we could do the following:

In [ ]:
for key in ['A', 'B', 'C']:
    print(key, df[df['key'] == key]['data'].sum())

This becomes very verbose when having multiple groups. You could make the above a bit easier by looping over the different values, but still, it is not very convenient to work with.

What we did above, applying a function on different groups, is a "groupby operation", and pandas provides some convenient functionality for this.

### Groupby: applying functions per group

The "group by" concept: we want to **apply the same function on subsets of your dataframe, based on some key to split the dataframe in subsets**

This operation is also referred to as the "split-apply-combine" operation, involving the following steps:

* **Splitting** the data into groups based on some criteria
* **Applying** a function to each group independently
* **Combining** the results into a data structure

<img src="./splitApplyCombine.png">

Similar to SQL `GROUP BY`

Instead of doing the manual filtering as above


    df[df['key'] == "A"].sum()
    df[df['key'] == "B"].sum()
    ...

pandas provides the `groupby` method to do exactly this:

In [ ]:
df.groupby('key').sum()

In [ ]:
df.groupby('key').aggregate(np.sum)  # 'sum'

And many more methods are available. 

In [ ]:
df.groupby('key')['data'].sum()

### Exercise: Application of the groupby concept on the titanic data

We go back to the titanic passengers survival data:

In [ ]:
df = pd.read_csv("data/titanic.csv")
df = df.set_index('Name')

In [ ]:
df.head()

* Using `groupby()`, calculate the average age for each sex.</li>


In [ ]:
# %load solutions/14_solutions.py

* Using the `groupby()` function, plot the age distribution for each sex.

In [ ]:
# %load solutions/15_solutions.py

* Plot the fare distribution based on the class.

In [ ]:
# %load solutions/16_solutions.py

* Plot the survival rate by class with a bar plot.

In [ ]:
# %load solutions/17_solutions.py

* Compute the survival rate grouping by class and sex. (Hint: you can pass a list to the `groupby` function)

In [ ]:
# %load solutions/18_solutions.py

## 6. Using `seaborn` for statistical analysis and plotting

We already used `matplotlib` and `pandas` to plot some information. `seaborn` offers some nice plotting capabilities to make statistical analysis with a minimum of preprocessing.

In [ ]:
import seaborn as sns

Check the documentation of `seaborn.catplot` and plot the survival rate grouping by the class and sex.

In [ ]:
# %load solutions/19_solutions.py

Sometime, one is intersted in seeing the distribution differences of some variables. The function `seaborn.boxplot` allows to plot such interaction. You can use this function to plot the `Fare` distribution splitting the data using the `Survived` and `Sex` variable.

In [ ]:
# %load solutions/20_solutions.py

The boxplot give information about the different quantiles. One can use the `seaborn.violinplot` to get the distribution information instead of the quantiles. Check the documentation of `seaborn.violinplot` and plot the same information as previously requested.

In [ ]:
# %load solutions/20_solutions.py

Another useful plotting tool is the function `seaborn.paiplot` which plot the interaction between all the different feature.

In [ ]:
sns.pairplot(df, hue='Survived')

However, this plotting method is better suited to visualize interaction for continuous feature and with a limited number of features. Let's check an example with the `iris` dataset.

In [ ]:
import seaborn as sns
sns.set(style="ticks")

df = sns.load_dataset("iris")
sns.pairplot(df, hue="species")

## 7. Data import and export

A wide range of input/output formats are natively supported by pandas:

* CSV, text
* SQL database
* Excel
* HDF5
* json
* html
* pickle
* sas, stata
* (parquet)
* ...

In [ ]:
# pd.read

In [ ]:
# df.to

Very powerful csv reader:

In [ ]:
pd.read_csv?

Luckily, if we have a well formed csv file, we don't need many of those arguments:

In [ ]:
df = pd.read_csv("data/titanic.csv")

In [ ]:
df.head()

<div class="alert alert-success">

<b>EXERCISE</b>: Read the `data/20000101_20161231-NO2.csv` file into a DataFrame `no2`
<br><br>
Some aspects about the file:
 <ul>
  <li>Which separator is used in the file?</li>
  <li>The second row includes unit information and should be skipped (check `skiprows` keyword)</li>
  <li>For missing values, it uses the `'n/d'` notation (check `na_values` keyword)</li>
  <li>We want to parse the 'timestamp' column as datetimes (check the `parse_dates` keyword)</li>
</ul>
</div>

In [ ]:
# %load solutions/22_solutions.py

## 7. Working with time series data

In [ ]:
no2 = pd.read_csv('data/20000101_20161231-NO2.csv', sep=';',
                  skiprows=[1], na_values=['n/d'],
                  index_col=0, parse_dates=True)

When we ensure the DataFrame has a `DatetimeIndex`, time-series related functionality becomes available:

In [ ]:
no2.index

Indexing a time series works with strings:

In [ ]:
no2["2010-01-01 09:00": "2010-01-01 12:00"]

A nice feature is "partial string" indexing, so you don't need to provide the full datetime string.

E.g. all data of January up to March 2012:

In [ ]:
no2['2012-01':'2012-03']

Time and date components can be accessed from the index:

In [ ]:
no2.index.hour

In [ ]:
no2.index.year

## The power of pandas: `resample`

A very powerfull method is **`resample`: converting the frequency of the time series** (e.g. from hourly to daily data).

Remember the air quality data:

In [ ]:
no2.plot()

The time series has a frequency of 1 hour. I want to change this to daily:

In [ ]:
no2.head()

In [ ]:
no2.resample('D').mean().head()

Above I take the mean, but as with `groupby` I can also specify other methods:

In [ ]:
no2.resample('D').max().head()

The string to specify the new time frequency: http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases  
These strings can also be combined with numbers, eg `'10D'`.

Further exploring the data:

In [ ]:
no2.resample('M').mean().plot() # 'A'

In [ ]:
no2.loc['2009':, 'VERS'].resample('M').agg(['mean', 'median']).plot()

### Exercise

The evolution of the yearly averages with, and the overall mean of all stations

* Use `resample` and `plot` to plot the yearly averages for the different stations.
* The overall mean of all stations can be calculated by taking the mean of the different columns (`.mean(axis=1)`).


In [ ]:
# %load solutions/23_solutions.py

## 8. What I didn't talk about

- Concatenating data: `pd.concat`
- Merging and joining data: `pd.merge`
- Reshaping data: `pivot_table`, `melt`, `stack`, `unstack`
- Working with missing data: `isnull`, `dropna`, `interpolate`, ...
- ...


## Further reading

* Pandas documentation: http://pandas.pydata.org/pandas-docs/stable/

* Books

    * "Python for Data Analysis" by Wes McKinney
    * "Python Data Science Handbook" by Jake VanderPlas

* Tutorials (many good online tutorials!)

  * https://github.com/jorisvandenbossche/pandas-tutorial
  * https://github.com/brandon-rhodes/pycon-pandas-tutorial

* Tom Augspurger's blog

  * https://tomaugspurger.github.io/modern-1.html